# Dashboards

In our last lecture we briefly looked over some of the web-based technologies for generating interactive elements. We specifically looked at `hvPlot` and `panel`. The former provides a separate but very similar plotting interface from `pandas` that provides us with many convenient add-ons. The latter allows us to create interactive widgets that we can use to control and modify our visualizations. Both of these tools are mostly built on top of `bokeh`, a web-based visualization library.

In this notebook we will look at more example of how we can start constructing basic dashboards with `bokeh`/`hvPlot`/`holoviews` and `panel`.

## Imports

First thing we need to so is import a few things to get started:

In [ ]:
import hvplot.pandas  # for plotting from data frames
import numpy as np  # for data generation
import pandas as pd  # for data frames
import panel as pn  # for interactive widgets

note that we do not need to import `bokeh` explicitly - `hvplot` and `panel` handle all of this for us! We may need to use `bokeh` directly, but for now we do not.

## Generating Data

Now we need to work with some data! We are just going to randomly generate some basic data that consists of a label, an id (x value) and a value (y value). We want data that is basic enough for a scatter plot, but also clearly groupable (in our case, the label). We also ensure that the x values across the entire set are already sorted and uniformly distributed.

In [ ]:
rng = np.random.RandomState(
    13337
)  # create state for random number generator for consistency
data = pd.DataFrame(
    {  # create a DataFrame from a dictionary
        "label": rng.choice(  # create the label column by randomly selecting values
            a=[
                "foo",
                "bar",
                "baz",
            ],  # from the list (given to a) 100 times, using the
            p=[0.5, 0.35, 0.15],  # probablities/weights given for each
            size=100,
        ),
        "id": np.arange(
            0, 100, 1
        ),  # create the id column by making a list of integers 0-99
        "value": rng.uniform(
            size=100
        ),  # create the value column by selecting 100 numbers between 0.0 and 1.0
    }
)
data

## Widgets

We are going to create some widgets that allow us to select labels of data and we will dynamically update our visualization with this data. We also want to provide a threshold filter (we saw something similar last class) to "disable" values. The widget we will use for selecting labels is called a `MultiSelect`, and the widget we will use for filtering values is called a `FloatSlider`.

In [ ]:
label_selector = pn.widgets.MultiSelect(  # create a MultiSelect widget
    name="Label",  # name of the widget
    options=["foo", "bar", "baz"],  # options for users to select from
    value=["foo", "bar", "baz"],  # default selections
)
threshold_slider = pn.widgets.FloatSlider(  # create a FloatSLider
    name="Threshold",  # name of the widget
    start=0.0,  # lower bound of the slider
    end=1.0,  # upper bound of the slider
    step=0.01,  # step value
    value=1.0,  # default value of the slider
)
label_selector

In [ ]:
threshold_slider

In subsequent cells below we are going to be recreating the the widgets in each cell that we would need them. As we will be make multiple plots withe same widgets, we do not want the rendered widgets from one cell to modify the plots from the other cells. This will happen because the state of the widgets will be tied to the state of the plots.

## Connecting Plots & Widgets

Now we need to create a function that plots our data. It needs to be a function because as the widgets change they need to reproduce the plot. They can only do this by giving it a set of instructions to carry out, which is exactly what a function is! There are a few different ways we can do this, but we are just going to lean into `panel` to handle this for us using the `interact` functionality.

The first thing we need to do is write a function that plots our data using values we expect to give it from our widgets. We have widgets for selecting labels and setting thresholds, and so our function should expect to take two variables - one for each of them. This function should build a mask from the two variables, apply it to the data frame, and then use `hvPlot` to scatter the data.

We are going to employ some details we saw last class to ensure that we anchor the limits of our axes.

In [ ]:
# first define a function that takes in placeholders for our widgets
def visualize_data(selections, threshold):
    # create the mask by checking which labels are in our selections
    # and which values are less than or equal to our threshold
    mask = (data.label.isin(selections)) & (data.value <= threshold)

    return (
        data[mask]
        .hvplot.scatter(  # apply the mask and scatter via hvPlot
            x="id",  # put the id on the x-axis
            y="value",  # put the value on the y-xis
            by="label",  # group by labels (i.e. colo by label)
        )
        .opts(  # further customize the plot via its options (opts)
            xlim=(0, 100),  # set the range of the x-axis to 0-100
            ylim=(0, 1),  # set the range of the y-axis to 0-1
        )
    )


# repeat our widgets (keeping them on 1 line each for brevity)
label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

# use panel to stitch our functions together with our widgets, mapping placeholders to widgets
interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)
interaction

The problem here is that if we deselect all of our labels, then not only is there nothing to plot, but `hvPLot` does not like it when it attempts to plot groups of data (as specified by `by='label'` and there is nothing to group. It is generally a good practice to check if there is actual data to plot, and in the case where there is nothing to plot we instead return a *pane* that says so.

In [ ]:
# updated function
def visualize_data(selections, threshold):
    mask = (data.label.isin(selections)) & (data.value <= threshold)

    # instead of just plotting here, we first check if the mask is too strict
    #
    # if the dataframe with the mask is empty, then return a message. otherwise
    # plot as we did before.
    if data[mask].empty:
        message = (
            "# No Data to Plot!\n### Perhaps you filtered too aggressively"
        )
        return pn.pane.Markdown(message)
    else:
        return (
            data[mask]
            .hvplot.scatter(x="id", y="value", by="label")
            .opts(xlim=(0, 100), ylim=(0, 1))
        )


label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)
interaction

A problem here, though this works, is that the colors are not consistent. As we develop interactive displays we need to pay attention to what details should be changing and what details actually are changing. Here `bokeh` (for better or worse) alphabetizes the labels used for grouping the data, and thus depending on the selections different colors are used for different labels.

Here we need to rework the scatter a bit, providing it with a color map that maps labels to specific colors. This means that as users provide different selections, the labels retain their specific colors.

In [ ]:
# create a dictionary that maps label names to colors
color_mapper = {"foo": "red", "bar": "green", "baz": "blue"}


# updeated function
def visualize_data(selections, threshold):
    mask = (data.label.isin(selections)) & (data.value <= threshold)
    if data[mask].empty:
        return pn.pane.Markdown(
            "# No Data to Plot!\n### Perhaps you filtered too aggressively"
        )
    else:
        return (
            data[mask]
            .hvplot.scatter(
                x="id",
                y="value",
                color="label",  # use the color specifier instead of by
                cmap=color_mapper,  # provide our dictionary as the colormapper
            )
            .opts(xlim=(0, 100), ylim=(0, 1))
        )


label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)
interaction

### Updating Over Replotting

The interactive plots we have produced above all work by applying masks to the data, plotting it, and then returning the plot. This sometimes can be inefficient, especially if producing the plot itself is computationally complex (perhaps due to large data). Instead of replotting, we can simply change what was plotted. This is the equivalent of *moving* paint and changing colors already on a canvas, rather than setting up a new canvas and painting the new image from scratch. Here the performance gains are not noticeable, but there are some really interesting things we can do with this later on.

To do this we need to first create out plot via `hvPlot` and save it to a variable. Once we have this variable we can then update its `data` attribute. So long as the new data satisfies all of the restrictions we put in place (e.g. names of columns that we specified) then `hvPlot`/`bokeh` takes care of the rest.

Observe below - we create a new variable `scatter` from our use of `hvPlot`, and then inside of `visualize_data` we then update `scatter.data` with the masked data frame before just returning the `scatter` object as it was.

In [ ]:
color_mapper = {"foo": "red", "bar": "green", "baz": "blue"}

# create our scatter our here and save it to a variable
scatter = data.hvplot.scatter(
    x="id", y="value", color="label", cmap=color_mapper
).opts(xlim=(0, 100), ylim=(0, 1))


# updated function
def visualize_data(selections, threshold):
    mask = (data.label.isin(selections)) & (data.value <= threshold)
    if data[mask].empty:
        return pn.pane.Markdown(
            "# No Data to Plot!\n### Perhaps you filtered too aggressively"
        )
    else:
        scatter.data = data[mask]  # update the data attribute of the scatter
        return scatter  # return the scatter


label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)
interaction

## Rearranging the Layout

The default layout from `panel` is very unlikely to be how you set out to create your interactive visualization. We can however rearrange how the widgets and the plots are all laid out. To understand how to do this we should inspect the `interaction` variable that we have been producing - we can do this by *printing* it instead of letting it render into its widgets and default layout.

In [ ]:
print(interaction)

By printing the interaction produced by `panel` we can actually see how `panel` lays everything out, and furthermore we can even see that it provides indices for accessing the individual components of the layout. For example let's look at the 0th element fo the 1st index:

In [ ]:
interaction[1][0]

It is our plot from above! If we modify the widgets above we will see both plots updated. Likewise if we accessed the widgets (in the 0th index) then interacting with them here would modify the plot above. What this ultimately means is that we can simply move our widgets into a new layout, into something that may make more sense. For this example we want to put the plot on top, with the widgets side-by-side underneath. For completeness (and to avoid too many plots updating all at once) we are going to include everything again.

We can use a combination of rows and columns to build everything out. We are also going to go a step further and add some additional text to the `panel` layout (similar to our error message when we over-filter):

In [ ]:
color_mapper = {"foo": "red", "bar": "green", "baz": "blue"}
scatter = data.hvplot.scatter(
    x="id", y="value", color="label", cmap=color_mapper
).opts(xlim=(0, 100), ylim=(0, 1))


def visualize_data(selections, threshold):
    mask = (data.label.isin(selections)) & (data.value <= threshold)
    if data[mask].empty:
        return pn.pane.Markdown(
            "# No Data to Plot!\n### Perhaps you filtered too aggressively",
            height=300,
            width=700,
        )
    else:
        scatter.data = data[mask]
        return scatter


label_selector = pn.widgets.MultiSelect(
    name="Label", options=["foo", "bar", "baz"], value=["foo", "bar", "baz"]
)
threshold_slider = pn.widgets.FloatSlider(
    name="Threshold", start=0.0, end=1.0, step=0.01, value=1.0
)

interaction = pn.interact(
    visualize_data, selections=label_selector, threshold=threshold_slider
)

# instead of just returning interaction, we are going to pull it a part a bit and put things back together

# we want to add a small header, so we first create some text for that header;
# it is going to be implemented as markdown for easy formatting
header = """### First Micro-dashboard

This micro-dashboard shows basic usage of panel's widgets and how we can tie them to visualizations. Select the labels you want to view and apply a threshold to filter values.
"""

# create the layout, we are going for something like
# _______________
# |___header____|
# |             |
# |    plot     |
# |_____________|
# |  w1  |  w2  |
# |______|______|
#
#
# This layout consists of a single column, with the bottom
# element consisting of a row of two elements.
#
# panel provides Row and Column layouts, just we stick them together
# nest them as we need to.
layout = pn.Column(
    pn.pane.Markdown(
        header
    ),  # add the header as a markdown pane at the top element in the column
    interaction[1],  # add our plot as the middle element in the column
    pn.Row(
        *interaction[0]
    ),  # add the widgets as a row as the bottow element in the column
    # ... we need to unpack the column and repack it as a Row
)
layout